# Neural Language Model using RNN

 - We can essentially build two kinds of language models – character level and word level. 
 - And even under each category, we can have many subcategories based on the simple fact of how we are framing the learning problem. 
 - We will be taking the most straightforward approach – building a character-level language model.



## Importing required libraries

In [1]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.
/Users/guptadhruv51/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/guptadhruv51/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/guptadhruv51/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/guptadhruv51/anaconda3/lib/p

## Dataset

<img src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/08/vc51.jpg" width=200 ></img>

 - Does the above text seem familiar? It’s the US Declaration of Independence! The dataset we will use is the text from this Declaration.

 - This is a historically important document because it was signed when the United States of America got independence from the British.

 - It’s the right size to experiment with because we are training a character-level language model which is comparatively more intensive to run as compared to a word-level language model.

 - The problem statement is to train a language model on the given text and then generate text given an input text in such a way that it looks straight out of this document and is grammatically correct and legible to read.

In [2]:
data_text = """The unanimous Declaration of the thirteen united States of America, When in the Course of human events, it becomes necessary for one people to dissolve the political bands which have connected them with another, and to assume among the powers of the earth, the separate and equal station to which the Laws of Nature and of Nature's God entitle them, a decent respect to the opinions of mankind requires that they should declare the causes which impel them to the separation.

We hold these truths to be self-evident, that all men are created equal, that they are endowed by their Creator with certain unalienable Rights, that among these are Life, Liberty and the pursuit of Happiness.--That to secure these rights, Governments are instituted among Men, deriving their just powers from the consent of the governed, --That whenever any Form of Government becomes destructive of these ends, it is the Right of the People to alter or to abolish it, and to institute new Government, laying its foundation on such principles and organizing its powers in such form, as to them shall seem most likely to effect their Safety and Happiness. Prudence, indeed, will dictate that Governments long established should not be changed for light and transient causes; and accordingly all experience hath shewn, that mankind are more disposed to suffer, while evils are sufferable, than to right themselves by abolishing the forms to which they are accustomed. But when a long train of abuses and usurpations, pursuing invariably the same Object evinces a design to reduce them under absolute Despotism, it is their right, it is their duty, to throw off such Government, and to provide new Guards for their future security.--Such has been the patient sufferance of these Colonies; and such is now the necessity which constrains them to alter their former Systems of Government. The history of the present King of Great Britain is a history of repeated injuries and usurpations, all having in direct object the establishment of an absolute Tyranny over these States. To prove this, let Facts be submitted to a candid world.

He has refused his Assent to Laws, the most wholesome and necessary for the public good.

He has forbidden his Governors to pass Laws of immediate and pressing importance, unless suspended in their operation till his Assent should be obtained; and when so suspended, he has utterly neglected to attend to them.

He has refused to pass other Laws for the accommodation of large districts of people, unless those people would relinquish the right of Representation in the Legislature, a right inestimable to them and formidable to tyrants only.

He has called together legislative bodies at places unusual, uncomfortable, and distant from the depository of their public Records, for the sole purpose of fatiguing them into compliance with his measures.

He has dissolved Representative Houses repeatedly, for opposing with manly firmness his invasions on the rights of the people.

He has refused for a long time, after such dissolutions, to cause others to be elected; whereby the Legislative powers, incapable of Annihilation, have returned to the People at large for their exercise; the State remaining in the mean time exposed to all the dangers of invasion from without, and convulsions within.

He has endeavoured to prevent the population of these States; for that purpose obstructing the Laws for Naturalization of Foreigners; refusing to pass others to encourage their migrations hither, and raising the conditions of new Appropriations of Lands.

He has obstructed the Administration of Justice, by refusing his Assent to Laws for establishing Judiciary powers.

He has made Judges dependent on his Will alone, for the tenure of their offices, and the amount and payment of their salaries.

He has erected a multitude of New Offices, and sent hither swarms of Officers to harrass our people, and eat out their substance.

He has kept among us, in times of peace, Standing Armies without the Consent of our legislatures.

He has affected to render the Military independent of and superior to the Civil power.

He has combined with others to subject us to a jurisdiction foreign to our constitution, and unacknowledged by our laws; giving his Assent to their Acts of pretended Legislation:

For Quartering large bodies of armed troops among us:

For protecting them, by a mock Trial, from punishment for any Murders which they should commit on the Inhabitants of these States:

For cutting off our Trade with all parts of the world:

For imposing Taxes on us without our Consent:

For depriving us in many cases, of the benefits of Trial by Jury:

For transporting us beyond Seas to be tried for pretended offences

For abolishing the free System of English Laws in a neighbouring Province, establishing therein an Arbitrary government, and enlarging its Boundaries so as to render it at once an example and fit instrument for introducing the same absolute rule into these Colonies:

For taking away our Charters, abolishing our most valuable Laws, and altering fundamentally the Forms of our Governments:

For suspending our own Legislatures, and declaring themselves invested with power to legislate for us in all cases whatsoever.

He has abdicated Government here, by declaring us out of his Protection and waging War against us.

He has plundered our seas, ravaged our Coasts, burnt our towns, and destroyed the lives of our people.

He is at this time transporting large Armies of foreign Mercenaries to compleat the works of death, desolation and tyranny, already begun with circumstances of Cruelty & perfidy scarcely paralleled in the most barbarous ages, and totally unworthy the Head of a civilized nation.

He has constrained our fellow Citizens taken Captive on the high Seas to bear Arms against their Country, to become the executioners of their friends and Brethren, or to fall themselves by their Hands.

He has excited domestic insurrections amongst us, and has endeavoured to bring on the inhabitants of our frontiers, the merciless Indian Savages, whose known rule of warfare, is an undistinguished destruction of all ages, sexes and conditions.

In every stage of these Oppressions We have Petitioned for Redress in the most humble terms: Our repeated Petitions have been answered only by repeated injury. A Prince whose character is thus marked by every act which may define a Tyrant, is unfit to be the ruler of a free people.

Nor have We been wanting in attentions to our Brittish brethren. We have warned them from time to time of attempts by their legislature to extend an unwarrantable jurisdiction over us. We have reminded them of the circumstances of our emigration and settlement here. We have appealed to their native justice and magnanimity, and we have conjured them by the ties of our common kindred to disavow these usurpations, which, would inevitably interrupt our connections and correspondence. They too have been deaf to the voice of justice and of consanguinity. We must, therefore, acquiesce in the necessity, which denounces our Separation, and hold them, as we hold the rest of mankind, Enemies in War, in Peace Friends.

We, therefore, the Representatives of the united States of America, in General Congress, Assembled, appealing to the Supreme Judge of the world for the rectitude of our intentions, do, in the Name, and by Authority of the good People of these Colonies, solemnly publish and declare, That these United Colonies are, and of Right ought to be Free and Independent States; that they are Absolved from all Allegiance to the British Crown, and that all political connection between them and the State of Great Britain, is and ought to be totally dissolved; and that as Free and Independent States, they have full Power to levy War, conclude Peace, contract Alliances, establish Commerce, and to do all other Acts and Things which Independent States may of right do. And for the support of this Declaration, with a firm reliance on the protection of divine Providence, we mutually pledge to each other our Lives, our Fortunes and our sacred Honor."""

## Preprocessing the data

### Cleaning

 - We perform basic text preprocessing since this data does not have much noise. 
 - We lower case all the words to maintain uniformity and remove words with length less than 3


In [3]:
import re

def text_cleaner(text):
    # lower case text
    newString = text.lower()
    newString = re.sub(r"'s\b","",newString)
    # remove punctuations
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    long_words=[]
    # remove short word
    for i in newString.split():
        if len(i)>=3:                  
            long_words.append(i)
    return (" ".join(long_words)).strip()


In [4]:
print("RAW TEXT:\n", data_text[:150])
# clean the text
data_new = text_cleaner(data_text)
# processed text
print("\nPROCESSED TEXT:\n", data_new[:150])

RAW TEXT:
 The unanimous Declaration of the thirteen united States of America, When in the Course of human events, it becomes necessary for one people to dissolv

PROCESSED TEXT:
 the unanimous declaration the thirteen united states america when the course human events becomes necessary for one people dissolve the political band


In [5]:
print("String length:", len(data_new))

String length: 7082


Once the preprocessing is complete, it is time to create training sequences for the model.

### Generating character sequences

- The way this problem is modeled is we take in 30 characters as context and ask the model to predict the next character. 
- Now, 30 is a number which I got by trial and error and you can experiment with it too. 
- You essentially need enough characters in the input sequence that your model is able to get the context.

In [6]:
def create_seq(text):
    length = 30
    sequences = list()
    for i in range(length, len(text)):
        # select sequence of tokens
        seq = text[i-length:i+1]
        # store
        sequences.append(seq)
    print('Total Sequences: %d' % len(sequences))
    return sequences

In [7]:
sequences = create_seq(data_new)
sequences[:5]

Total Sequences: 7052


['the unanimous declaration the t',
 'he unanimous declaration the th',
 'e unanimous declaration the thi',
 ' unanimous declaration the thir',
 'unanimous declaration the thirt']

Once the sequences are generated, the next step is to encode each character. This would give us a sequence of numbers.


### Numerical encoding of characters

In [8]:
# create a character mapping index
chars = sorted(list(set(data_new)))
mapping = dict((c, i) for i, c in enumerate(chars))

def encode_seq(seq):
    sequences = list()
    for line in seq:
        # integer encode line
        encoded_seq = [mapping[char] for char in line]
        # store
        sequences.append(encoded_seq)
    return sequences

# encode the sequences
sequences = encode_seq(sequences)

In [9]:
sequences[:5]


[[20,
  8,
  5,
  0,
  21,
  14,
  1,
  14,
  9,
  13,
  15,
  21,
  19,
  0,
  4,
  5,
  3,
  12,
  1,
  18,
  1,
  20,
  9,
  15,
  14,
  0,
  20,
  8,
  5,
  0,
  20],
 [8,
  5,
  0,
  21,
  14,
  1,
  14,
  9,
  13,
  15,
  21,
  19,
  0,
  4,
  5,
  3,
  12,
  1,
  18,
  1,
  20,
  9,
  15,
  14,
  0,
  20,
  8,
  5,
  0,
  20,
  8],
 [5,
  0,
  21,
  14,
  1,
  14,
  9,
  13,
  15,
  21,
  19,
  0,
  4,
  5,
  3,
  12,
  1,
  18,
  1,
  20,
  9,
  15,
  14,
  0,
  20,
  8,
  5,
  0,
  20,
  8,
  9],
 [0,
  21,
  14,
  1,
  14,
  9,
  13,
  15,
  21,
  19,
  0,
  4,
  5,
  3,
  12,
  1,
  18,
  1,
  20,
  9,
  15,
  14,
  0,
  20,
  8,
  5,
  0,
  20,
  8,
  9,
  18],
 [21,
  14,
  1,
  14,
  9,
  13,
  15,
  21,
  19,
  0,
  4,
  5,
  3,
  12,
  1,
  18,
  1,
  20,
  9,
  15,
  14,
  0,
  20,
  8,
  5,
  0,
  20,
  8,
  9,
  18,
  20]]

In [10]:
# vocabulary size
vocab = len(mapping)
vocab

27

In [11]:
mapping

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

### Train and validation sets

Once we are ready with our sequences, we split the data into training and validation splits. 

In [12]:
from sklearn.model_selection import train_test_split

# vocabulary size
vocab = len(mapping)
sequences = np.array(sequences)
# create X and y
X, y = sequences[:,:-1], sequences[:,-1]
# one hot encode y
y = to_categorical(y, num_classes=vocab)
# create train and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)

Train shape: (5641, 30) Val shape: (1411, 30)


In [13]:
X_tr.shape, X_val.shape

((5641, 30), (1411, 30))

## GRU Model

Time to build our language model!

 - We have used the embedding layer of Keras to learn a 50 dimension embedding for each character. 
 - This helps the model in understanding complex relationships between characters. 
 - The there is a GRU layer as the base model, which has 150 timesteps.
 -  Finally, a Dense layer is used with a softmax activation for prediction.

In [15]:
# define model
model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable=True))
model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
model.add(Dense(vocab, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 50)            1350      
_________________________________________________________________
gru_2 (GRU)                  (None, 150)               90450     
_________________________________________________________________
dense_2 (Dense)              (None, 27)                4077      
Total params: 95,877
Trainable params: 95,877
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)  
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True,verbose=1)  

In [17]:
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
model.fit(X_tr, y_tr, epochs=100, verbose=2, validation_data=(X_val, y_val))

W1123 12:57:57.256293 4521934336 deprecation_wrapper.py:119] From /Users/guptadhruv51/anaconda3/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1123 12:57:57.328624 4521934336 deprecation_wrapper.py:119] From /Users/guptadhruv51/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W1123 12:57:57.527123 4521934336 deprecation.py:323] From /Users/guptadhruv51/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 5641 samples, validate on 1411 samples
Epoch 1/100
 - 10s - loss: 2.8113 - acc: 0.1909 - val_loss: 2.4704 - val_acc: 0.2899
Epoch 2/100
 - 7s - loss: 2.3239 - acc: 0.3209 - val_loss: 2.2500 - val_acc: 0.3345
Epoch 3/100
 - 7s - loss: 2.1883 - acc: 0.3547 - val_loss: 2.1524 - val_acc: 0.3629
Epoch 4/100
 - 7s - loss: 2.0964 - acc: 0.3794 - val_loss: 2.0783 - val_acc: 0.3841
Epoch 5/100
 - 7s - loss: 2.0207 - acc: 0.3996 - val_loss: 2.0350 - val_acc: 0.4040
Epoch 6/100
 - 7s - loss: 1.9465 - acc: 0.4198 - val_loss: 1.9594 - val_acc: 0.4281
Epoch 7/100
 - 7s - loss: 1.8751 - acc: 0.4347 - val_loss: 1.9377 - val_acc: 0.4337
Epoch 8/100
 - 7s - loss: 1.8158 - acc: 0.4513 - val_loss: 1.8923 - val_acc: 0.4458
Epoch 9/100
 - 10s - loss: 1.7418 - acc: 0.4712 - val_loss: 1.8699 - val_acc: 0.4479
Epoch 10/100
 - 8s - loss: 1.6754 - acc: 0.4926 - val_loss: 1.8543 - val_acc: 0.4486
Epoch 11/100
 - 7s - loss: 1.6062 - acc: 0.5118 - val_loss: 1.8469 - val_acc: 0.4685
Epoch 12/100
 - 7s - los

Epoch 97/100
 - 7s - loss: 0.2961 - acc: 0.9048 - val_loss: 3.2344 - val_acc: 0.4543
Epoch 98/100
 - 6s - loss: 0.3045 - acc: 0.8959 - val_loss: 3.1820 - val_acc: 0.4621
Epoch 99/100
 - 6s - loss: 0.2966 - acc: 0.9023 - val_loss: 3.2135 - val_acc: 0.4649
Epoch 100/100
 - 6s - loss: 0.3024 - acc: 0.8986 - val_loss: 3.1953 - val_acc: 0.4727


## Inference - Generating output sequence

Once the model has finished training, we can generate text from the model given an input sequence using the below code:



In [18]:
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
            # append to input
        in_text += char
    return in_text

## Output

Let's see what is the output of our model!

In [19]:
inp = 'large armies of'
print(len(inp))
print(generate_seq(model, mapping, 30, inp.lower(), 30))

15
large armies offices and the same absolute ru
